## TODO
1. Speed up to_datetime calls
1. Saved mapped dataframes to file
1. Crime data
1. Fix mapping
    - Cartopy?

# Data Cleaning
## Overview
- Load in data used in the analyses
- Understand the structure, granularity, and quality of the data
- Clean up the data for further inspection

## Datasets
1. Oakland air quality data
1. Oakland neighborhoods
    - GeoJSON format data to plot different neighborhoods in Oakland. The idea here is to be able to aggregate any values of interest (e.g., air quality) by easily-identifiable neighborhoods.
1. Residential zones within 300 feet of industrial zones.
    - GeoJSON data that shows areas in which residents live very close to industrial zones. The purpose for doing this is that we expect to see increased pollution levels near these zones.
1. Oakland city service requests
    - CSV file containing requests (e.g., potholes, illegal dumping, etc.) from residents.

In [1]:
# Standard tools for data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Tools specific for geospatial data analysis
from shapely.geometry import shape, mapping, Point, Polygon
import geopandas as gpd
import geojsonio

# Tools from the Python Standard Library
import os
import re

from IPython.display import display
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

First, let's take a look at files that are available:

In [2]:
DATADIR = '../data/'
!ls $DATADIR

CrimeWatch_Maps_Past_90-Days.csv
Data_Delivery_EDF_AQ_Team_20170515.xlsx
Service_requests_received_by_the_Oakland_Call_Center.csv
oakland_neighborhoods.geojson
residential_zones_300_ft_of_industrial_areas.geojson


## 1. Oakland Air Quality Data
This is the centerpiece of the following analyses, so let's check this out first. It's in an excel spreadsheet file with multiple worksheets, so let's read all of those separately.

In [4]:
air_quality_xlsx = pd.ExcelFile(DATADIR + 'Data_Delivery_EDF_AQ_Team_20170515.xlsx')

In [5]:
air_quality_xlsx.sheet_names

['WestOak', 'Downtown', 'EastOak', 'Highways']

We have separate sheets for West Oakland, Downtown, East Oakland, and near Highways. For convenience, I can load all of these into a single DataFrame and add a column which represents which region the data are coming from. The reason for doing this is that it can ensure that we perform the same operations on all data.

In [6]:
air_quality = pd.DataFrame()

for region in air_quality_xlsx.sheet_names:
    # Create a temporary DataFrame
    df = air_quality_xlsx.parse(region)
    
    # Drop the Terms_of_Use column. Note that this is included in the root README.md. Please read this if you
    # are going to use the data further.
    df.drop(columns='Terms_of_Use', inplace=True)
    
    # Add the region to this DataFrame
    df['region'] = region
    
    # Now add this to a single large DataFrame
    air_quality = air_quality.append(df)

Let's take a look at the resulting DataFrame:

In [7]:
display(air_quality.head())
display(air_quality.tail())

,Longitude,Latitude,Value,Pollutant,region
0,-122.322594,37.806781,24.000000,NO,WestOak
1,-122.322310,37.806150,19.775000,NO,WestOak
2,-122.322301,37.806420,19.974999,NO,WestOak
3,-122.322299,37.805880,15.208696,NO,WestOak
4,-122.322267,37.806689,26.887681,NO,WestOak


,Longitude,Latitude,Value,Pollutant,region
18202,-122.034943,37.560076,3.923761,BC,Highways
18203,-122.034724,37.560164,2.133266,BC,Highways
18204,-122.034681,37.559830,2.659885,BC,Highways
18205,-122.034504,37.559958,1.888058,BC,Highways
18206,-122.034503,37.559957,2.117248,BC,Highways


This DataFrame yields the latitude,  longitude, and the measured value of particular pollutants in various regions around Oakland. The values for pollutant concentrations have been aggregated over many measurements under various conditions to ideally prevent from biasing towards ephemeral environmental conditions.

In [8]:
air_quality.describe()

,Longitude,Latitude,Value
count,68547.000000,68547.000000,68547.000000
mean,-122.216190,37.769863,13.229003
std,0.065577,0.051888,17.843548
min,-122.322594,37.559830,-0.669949
25%,-122.276867,37.745058,0.958451
50%,-122.200047,37.775018,6.827747
75%,-122.167626,37.809773,18.739774
max,-122.034503,37.837491,758.456155


Let's save this to our results directory for further use:

In [17]:
RESULTSDIR = '../results/'
if not os.path.exists(RESULTSDIR):
    os.mkdir(RESULTSDIR)

air_quality.to_hdf(RESULTSDIR + '01-air_quality.h5', 'air_quality')

## 2. Oakland neighborhood classification
- A GeoJSON file classifying the areas in Oakland into neighborhoods was pulled from https://data.oaklandnet.com/Property/Oakland-Neighborhoods/7zky-kcq9 on 22/11/17.
- The downloaded file was renamed, replacing spaces with underscores, and setting all letters to lowercase for convenience.

Let's read in this GeoJSON file:

### Note
It wasn't trivial to get this working! I spent quite some time hacking to get this to come out right (ultimately, it just came down to finding the right versions to use). Reading GeoJSON into Geopandas DataFrames should be straightforward, but I was met with quite a few errors. After doing some tracking, I found that the right combination was using installing `shapely` and `geopandas` from `pip` instead of `conda-forge`.

In [3]:
neighborhoods = gpd.read_file(DATADIR + 'oakland_neighborhoods.geojson')

In [4]:
display(neighborhoods.head())
display(neighborhoods.tail())

,name,description,geometry
0,Acorn/ Acorn Industrial,,POINT (-122.308714010677 37.802146145243)
1,Adams Point,,POINT (-122.254661526017 37.8121177617939)
2,Allendale,,POINT (-122.203566466228 37.7871405525981)
3,Arroyo Viejo,,POINT (-122.178464360722 37.7600613799457)
4,Bancroft Business/ Havenscourt,,POINT (-122.184941350471 37.7669285553085)


,name,description,geometry
269,Upper Rockridge,,"(POLYGON ((-122.219555006624 37.832981772973, ..."
270,Waverly,,"(POLYGON ((-122.261262247923 37.811458543436, ..."
271,Webster,,"(POLYGON ((-122.165951274503 37.7532857075336,..."
272,Woodland,,"(POLYGON ((-122.177602684022 37.7536107490519,..."
273,Woodminster,,"(POLYGON ((-122.178433674265 37.8037393938781,..."


In [ ]:
# Run this code if you'd like to check out the neighborhoods in an interactive browser
geojsonio.display(neighborhoods.to_json())

Let's check these neighborhoods out!

In [ ]:
f, ax = plt.subplots()
neighborhoods.plot(ax=ax);

### Maps

Notice how this just gives us the shapes of our neighborhoods without any context. Let's plot this on top of a map so we get an idea of where these are located with respect to other features we are familiar with. To do this, we will use `basemap`. It's worth noting that `basemap` is being replaced with `cartopy`, but since I'm somewhat familiar with `basemap` for the time being :).

We will use the same coordinates from the above plot to get our $x$ and $y$ limits. In `basemap`, these will be referred to as the lower (upper) left (right) corner latitude (longitude) or llcrnlat, etc.

As before, let's save this for later:

## 3. Residential areas within 300 feet of industrial zones
- A GeoJSON file classifying the areas in Oakland into neighborhoods was pulled from https://data.oaklandnet.com/Economic-Development/Residential-Zones-300-ft-of-Industrial-Areas/d3re-jdqr on 22/11/17.
- The downloaded file was renamed, replacing spaces with underscores, and setting all letters to lowercase for convenience.

Let's read in this GeoJSON file:

In [12]:
residential_industrial = gpd.read_file(DATADIR + 'residential_zones_300_ft_of_industrial_areas.geojson')

In [13]:
residential_industrial.head()

,area,fid_indust,basezone,perimeter,znlabel,overlay,lastupdate,id,fid_reside,ordinance,geometry
0,165781.541327,0,RM-1,1726.61174,RM-1,,20150108,0,0,,(POLYGON ((-122.1749871795865 37.7318118929050...
1,31291.800394,0,RM-3,831.336232,RM-3,,20150108,0,1,,(POLYGON ((-122.1865568615996 37.7372242278911...
2,41449.399938,0,RM-4,916.621191,RM-4,,20150108,0,3,,(POLYGON ((-122.186420524489 37.73760424195443...
3,18517341.381599,0,RD-1,29878.462383,RD-1,,20150108,0,6,,(POLYGON ((-122.1909303995534 37.7376543702245...
4,4290422.952443,0,RD-1,11371.141784,RD-1,,20150108,0,8,,(POLYGON ((-122.1749888306131 37.7392570579501...


In [14]:
# Run this code if you'd like to check out the neighborhoods in an interactive browser
geojsonio.display(residential_industrial.to_json())

'http://geojson.io/#id=gist:/5c2204bb1575c9da732f49ffcaa97b0d'

Let's overlay these zones on the neighborhood map:

In [ ]:
residential_industrial.plot(color='r', ax=ax);
display(f)

## 4. City Service Requests
- An Excel Spreadsheet containing service requests to the city was pulled on the evening of 22 November 2017. This record is regularly updated and contains the following information of interest:
    1. Request ID
    1. Date and time at which the request was registered in the system
    1. Source of service request (i.e., call, email, website)
    1. Description
    1. Request category (e.g., GRAFFITI)
    1. Request location (address and/or GPS coordinates)
    1. Status (open, closed, cancelled, etc.)
    1. Date and time at which the request was closed in the system
- The purpose for considering this data is to look at the frequency at which requests are made in different neighborhoods, as well as the distribution of times it takes to resolve issues.
- Now that I think about it, this is quite a rich dataset in itself, and could be the basis of this project alone!

In [5]:
service_requests = pd.read_csv(DATADIR + 'Service_requests_received_by_the_Oakland_Call_Center.csv')

In [6]:
service_requests.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT
0,770583,11/01/2017 06:25:49 AM,SeeClickFix,Park - Lighting,BLDGMAINT,SOUTH PRESCOTT PARK AKA CRESCENT PARK (1551-16...,OPEN,NaN,NaN,6042512.94,2119761.93,CCD3,02Y
1,770584,11/01/2017 06:34:04 AM,SeeClickFix,Parking - Enforcement,TRAFFIC_ENGIN,"E 23RD ST &amp; HUGHES AV\nOakland, CA\n(37.78...",OPEN,NaN,NaN,6064305.00,2112881.25,CCD5,21Y
2,770585,11/01/2017 07:22:20 AM,SeeClickFix,"Graffiti on Street, Street Light, Traffic Signal,",GRAFFITI,"280 LEE ST\nOakland, CA\n(37.812325, -122.256729)",WOCREATE,NaN,NaN,6054236.62,2122946.45,CCD3,14X
3,770586,11/01/2017 07:29:17 AM,Website,Litter - Street Litter Container - Overflowing...,ILLDUMP,"3150 NICOL AV\nOakland, CA\n(37.791987, -122.2...",OPEN,NaN,NaN,6066555.08,2115444.93,CCD5,21Y
4,770587,11/01/2017 07:29:18 AM,Website,Litter - Street Litter Container - Overflowing...,ILLDUMP,"3150 NICOL AVE\nOakland, CA\n(37.791987, -122....",CANCEL,NaN,NaN,0.00,0.00,NaN,NaN


For now, I'll drop the `REFERREDTO`, `SRX`, `SRY`, `COUNCILDISTRICT`, and `BEAT` columns.

In [7]:
service_requests.drop(columns=['REFERREDTO', 'SRX', 'SRY', 'COUNCILDISTRICT', 'BEAT'], inplace=True)

In [8]:
service_requests.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,DATETIMECLOSED
0,770583,11/01/2017 06:25:49 AM,SeeClickFix,Park - Lighting,BLDGMAINT,SOUTH PRESCOTT PARK AKA CRESCENT PARK (1551-16...,OPEN,NaN
1,770584,11/01/2017 06:34:04 AM,SeeClickFix,Parking - Enforcement,TRAFFIC_ENGIN,"E 23RD ST &amp; HUGHES AV\nOakland, CA\n(37.78...",OPEN,NaN
2,770585,11/01/2017 07:22:20 AM,SeeClickFix,"Graffiti on Street, Street Light, Traffic Signal,",GRAFFITI,"280 LEE ST\nOakland, CA\n(37.812325, -122.256729)",WOCREATE,NaN
3,770586,11/01/2017 07:29:17 AM,Website,Litter - Street Litter Container - Overflowing...,ILLDUMP,"3150 NICOL AV\nOakland, CA\n(37.791987, -122.2...",OPEN,NaN
4,770587,11/01/2017 07:29:18 AM,Website,Litter - Street Litter Container - Overflowing...,ILLDUMP,"3150 NICOL AVE\nOakland, CA\n(37.791987, -122....",CANCEL,NaN


Just from looking at the first five entries, we can see that we may run into mutiple entries! Requests 770586 and 770587 appear to be the same events. Let's get an idea of what the columns contain:

In [ ]:
display(service_requests['REQCATEGORY'].unique())
display(service_requests['SOURCE'].unique())
display(service_requests['STATUS'].unique())

Let's take a look at the unfunded requests:

In [ ]:
service_requests[service_requests['STATUS'] == 'UNFUNDED'].head()

We will want the GPS coordinates for easier plotting later on. Let's do some regex-ing to clean those addresses/coordinates up:

In [ ]:
service_requests['REQADDRESS'].head()

Here's a regex expression that can be used to pull GPS coordinates from the address column:

In [ ]:
re.findall('[\d.]+, [\-\d.]+', service_requests['REQADDRESS'].iloc[3])

We can use a function to perform this search and also return a tuple of GPS coordinates for us:

In [13]:
def get_coords(addr):
    coords = re.findall('[\d.]+, [\-\d.]+', addr)
    
    # Return a missing value if there aren't any coordinates
    if not coords:
        return np.nan

    # Return a tuple otherwise
    coords = coords[0].split(',')
    return (float(coords[0]), float(coords[1]))

In [ ]:
get_coords(service_requests['REQADDRESS'].iloc[1])

In [ ]:
service_requests['coordinates'] = service_requests['REQADDRESS'].apply(get_coords)

In [ ]:
service_requests.head()

In [ ]:
service_requests.loc[1]

In [ ]:
print('Number of entries with coordinates:', service_requests[service_requests['coordinates'].notnull()].shape[0])
print('Number of entries without coordinates:', service_requests[service_requests['coordinates'].isna()].shape[0])

Let's limit ourselves to the entries that have coordinates for us to plot (using the addresses provided is completely possible, but beyond the scope of our current analysis). As we see above, we still have a good amount of data to play with. It's worth noting that selecting only those with GPS coordinates may in fact be introducing a bias in our results, or it may in fact be a negiligble effect (e.g., the people responsible for data entry may not have entered the coordinates...).

In [ ]:
service_requests = service_requests[service_requests['coordinates'].notnull()]

In [ ]:
service_requests.head()

And since we are not using the address, let's just drop that column:

In [ ]:
service_requests.drop(columns='REQADDRESS', inplace=True)

Another piece of data that may be of interest to us is how long it took to close the request. For example, we can geospatially map out how long it took to close requests. To do this, let's first make sure the times are all in a common format.

In [12]:
service_requests.loc[:, 'DATETIMEINIT'] = pd.to_datetime(service_requests['DATETIMEINIT'], infer_datetime_format=True)
service_requests.loc[:, 'DATETIMECLOSED'] = pd.to_datetime(service_requests['DATETIMECLOSED'], infer_datetime_format=True)

What's the range of the dates here?

In [16]:
service_requests['DATETIMEINIT'].min(), service_requests['DATETIMEINIT'].max()

(Timestamp('2009-07-01 08:05:36'), Timestamp('2017-11-22 01:11:19'))

In [ ]:
service_requests.loc[:, 'time_to_close'] = service_requests['DATETIMECLOSED'] - service_requests['DATETIMEINIT']

In [ ]:
service_requests.sort_values(by='time_to_close', ascending=False).head(10)

We see that sometimes it takes years to close these! However, these are just the ones that are still open. If a request had never been closed, its value for `DATETIMECLOSED` will be `NaT` (i.e., not a time). As a result, we probably want a variable that gives us the time since opening. To do this, we can find the difference in time from when this was downloaded and when the request was opened.

In [ ]:
# This dataset was downloaded on 22 Nov. 2017
t_0 = pd.datetime(2017, 11, 22)

In [ ]:
service_requests.loc[:, 'time_since_init'] = t_0 - service_requests['DATETIMEINIT']

In [ ]:
service_requests.head()

We are in a good state to run further analyses on these data. Let's save it for further inspection:

In [ ]:
service_requests.to_hdf(RESULTSDIR + '01-service_requests.h5', 'service_requests')

Let's get some memory back now:

In [ ]:
del service_requests

## 5. Crime data
- Below we read in crimes reported to the Oakland Police Department in the 90 days prior to it's download (on 27 November).

In [4]:
crime = pd.read_csv(DATADIR + 'CrimeWatch_Maps_Past_90-Days.csv')

In [5]:
crime.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location
0,VANDALISM,10/24/2017 07:00:00 PM,17-916879,VANDALISM,NaN,500 20TH ST,Oakland,CA,"500 20TH ST\nOakland, CA\n(37.809581, -122.269..."
1,VANDALISM,09/08/2017 07:00:00 PM,17-914153,VANDALISM,08X,500 27TH ST,Oakland,CA,"500 27TH ST\nOakland, CA\n(37.816128, -122.267..."
2,MOTOR VEHICLE THEFT,10/06/2017 11:27:00 AM,17-052428,VEHICLE THEFT - AUTO,22Y,3300 GEORGIA ST,Oakland,CA,"3300 GEORGIA ST\nOakland, CA\n(37.794807, -122..."
3,ASSAULT,09/09/2017 11:16:00 PM,17-047455,INFLICT CORPORAL INJURY ON SPOUSE/COHABITANT,27Y,5900 HARMON AV,Oakland,CA,"5900 HARMON AV\nOakland, CA\n(37.768042, -122...."
4,ASSAULT,09/16/2017 04:37:00 PM,17-048647,BATTERY:SPOUSE/EX SPOUSE/DATE/ETC,35X,2400 96TH AV,Oakland,CA,"2400 96TH AV\nOakland, CA\n(37.750884, -122.16..."


Let's drop `POLICEBEAT`, `ADDRESS`, `CITY`, and `STATE` since we won't need those. As before, we can pull the geolocation in terms of latitude and longitude from `Location`.

In [11]:
crime.drop(['POLICEBEAT', 'ADDRESS', 'CITY', 'STATE'], axis=1, inplace=True)

In [12]:
crime.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,Location
0,VANDALISM,10/24/2017 07:00:00 PM,17-916879,VANDALISM,"500 20TH ST\nOakland, CA\n(37.809581, -122.269..."
1,VANDALISM,09/08/2017 07:00:00 PM,17-914153,VANDALISM,"500 27TH ST\nOakland, CA\n(37.816128, -122.267..."
2,MOTOR VEHICLE THEFT,10/06/2017 11:27:00 AM,17-052428,VEHICLE THEFT - AUTO,"3300 GEORGIA ST\nOakland, CA\n(37.794807, -122..."
3,ASSAULT,09/09/2017 11:16:00 PM,17-047455,INFLICT CORPORAL INJURY ON SPOUSE/COHABITANT,"5900 HARMON AV\nOakland, CA\n(37.768042, -122...."
4,ASSAULT,09/16/2017 04:37:00 PM,17-048647,BATTERY:SPOUSE/EX SPOUSE/DATE/ETC,"2400 96TH AV\nOakland, CA\n(37.750884, -122.16..."


In [15]:
crime['coordinates'] = crime['Location'].apply(get_coords)

In [16]:
crime.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,Location,coordinates
0,VANDALISM,10/24/2017 07:00:00 PM,17-916879,VANDALISM,"500 20TH ST\nOakland, CA\n(37.809581, -122.269...","(37.809581, -122.269628)"
1,VANDALISM,09/08/2017 07:00:00 PM,17-914153,VANDALISM,"500 27TH ST\nOakland, CA\n(37.816128, -122.267...","(37.816128, -122.267219)"
2,MOTOR VEHICLE THEFT,10/06/2017 11:27:00 AM,17-052428,VEHICLE THEFT - AUTO,"3300 GEORGIA ST\nOakland, CA\n(37.794807, -122...","(37.794807, -122.20291)"
3,ASSAULT,09/09/2017 11:16:00 PM,17-047455,INFLICT CORPORAL INJURY ON SPOUSE/COHABITANT,"5900 HARMON AV\nOakland, CA\n(37.768042, -122....","(37.768042, -122.194877)"
4,ASSAULT,09/16/2017 04:37:00 PM,17-048647,BATTERY:SPOUSE/EX SPOUSE/DATE/ETC,"2400 96TH AV\nOakland, CA\n(37.750884, -122.16...","(37.750884, -122.160601)"


Neat. Let's save this to use HDF5 for later use.

In [19]:
crime.to_hdf(RESULTSDIR + '01-crime.h5', 'crime')

/Users/kjbilton/Installs/miniconda2/envs/oakland/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['CRIMETYPE', 'DATETIME', 'CASENUMBER', 'DESCRIPTION', 'Location', 'coordinates']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
